### 作業目的: 使用N-Gram模型預測文字

本次作業會使用[桃園市官網市政新聞](https://data.gov.tw/dataset/25891)來進行練習

### 載入所需的Libraries

In [1]:
import json
import re
from collections import Counter, namedtuple

---
### 載入資料

In [2]:
# 使用 json.load()

# news_data = json.load(open('./WebNews.json', 'r'))
# news_data[:20]

In [3]:
with open('./WebNews.json', 'r') as f:
    news_data = json.load(f)

In [4]:
type(news_data)

list

In [5]:
news_data[:2]

[{'file': [],
  'link': [],
  'id': '202006110003',
  'istop': 'N',
  'img': [{'imgname': '0611桃園屏東_200611_0004.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>與<br>潘<br>縣<br>長<br>合<br>體<br>推<br>廣<br>防<br>疫<br>互<br>惠<br>旅<br>遊<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817400.jpg&filedisplay=202006111817400.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0005.jpg',
    'imgcontent': '<br>潘<br>縣<br>長<br>挑<br>選<br>屏<br>東<br>特<br>色<br>伴<br>手<br>禮<br>致<br>贈<br>鄭<br>市<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817401.jpg&filedisplay=202006111817401.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0007.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>精<br>心<br>準<br>備<br>桃<br>園<br>特<br>色<br>伴<br>手<br>禮<br>送<br>給<br>潘<br>縣<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817402.jpg&filedisplay=202006111817402.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0006.jpg',
    'imgcontent': '<br>桃<br>園<br

---
### 進行資料清洗
觀察上面的資料，資料包含許多其他的資訊，我們需要將真正的新聞內文取出，並且對內文進行文字清洗。
請做以下的文字處理:

1. 取出新聞內文
2. 去除HTML Tags
3. 移除標點符號，只保留英文、數字、中文

In [6]:
#取出新聞內文
corpus_list = [content['detailcontent'] for content in news_data]

In [7]:
#去除HTML Tags與標點符號(只保留英文、數字、中文)
corpus_list = list([''.join(re.findall(r'[\u4E00-\u9FA50-9]', content)) for content in corpus_list])
corpus_list[0]

'迎接國旅爆發期五星縣市長合體推廣桃園屏東互惠旅遊桃園市長鄭文燦今11日下午出席桃園加屏東旅遊優惠強棒一棒接一棒好康發表記者會鄭市長表示隨著疫情趨緩國內進入國旅爆發期觀光局近期公布最新10大熱門旅遊景點桃園包辦前三名依序為大溪老街石門水庫及角板山顯示桃園旅遊的潛力無限鄭市長說桃園市與屏東縣各自擁有獨特的觀光魅力與資源希望能搭配中央觀光振興方案共同推廣防疫互惠旅遊並推出更多加碼優惠措施降低民眾的旅遊負擔增加旅遊的樂趣也讓國境之南與國境之門做最好的交流鄭市長提到桃園是國門之都屏東是國境之南兩個城市各有特色及魅力許多景點也互相輝映假如你喜歡屏東的山川琉璃吊橋絕對不能錯過桃園的新溪口吊橋如果你喜歡大溪老街也應該去恆春古城漫遊鄭市長說許多民眾造訪過很多國外景點例如普吉島巴里島卻沒有到過小琉球澎湖等離島十分可惜他也稱讚屏東幅員遼闊擁有多樣化的景點墾丁更是國內旅遊勝地之一希望現階段推動防疫新生活的同時桃園也能與各縣市合作一起推廣防疫互惠旅遊讓更多在地旅遊業者受惠桃園預估下半年包含鄰里長環保志工守望相助隊義警民防等觀摩旅遊團數出團數將達500團以上目前桃園第一階段已經與基隆台南屏東等縣市簽署合作契約第二階段將與宜蘭花蓮南投嘉義等縣市合作希望未來逐漸擴大至其他縣市提供遊客更多優惠措施鄭市長也說屏東縣推出三鐵優惠方案桃園市自7月1日起推出來桃園住一晚加贈免費一日遊方案遊客來桃園在合法旅宿住一晚即可獲得市府加贈免費一日遊行程內容包含遊覽車接送午餐景點門票或體驗等期盼帶領遊客體驗桃園的觀光魅力此外民眾也可以持200元振興三倍券兌換400元夜市券藉此鼓勵民眾到夜市消費市府未來也將陸續推出更多的加碼優惠措施屏東縣長潘孟安則希望在鄭市長推廣下吸引更多桃園市民到屏東旅遊他表示桃園為國境之門擁有很大的發展優勢在鄭市長帶領下桃園連續多年結婚率及出生率都是全國之冠也成為一座宜居城市他也特別推薦親子旅遊首選屏東屏東縣政府整合轄內旅宿業者伴手禮業者食品業者等推出鐵定加碼鐵定貼心鐵定好玩三鐵優惠7月15日以前遊客在恆春地區住宿一晚即加贈場館門票或等值餐飲券在東港地區住宿則可用10元加購伴手禮及小王馬福袋10人以上團體旅遊每人可獲得500元旅費補助每團上限1萬元屏東也有許多當季優質農特產品及特色伴手禮歡迎桃園市民踴躍到屏東旅遊保證遊客玩得痛快買得愉快吃得爽快全國中等學校運動會今109年首度移師屏東舉辦潘縣長也特

---
### 建立N-Gram模型
N-Gram模型在計算詞機率時為(以Trigram為例)
$$
P(W_i|W_{i-1},W_{i-2}) = \frac{count(W_i, W_{i-1}, W_{i-2})}{count(W_{i-1}, W_{i-2})}
$$

舉例來說
$$
P(the|this,is) = \frac{count(this\ is\ the)}{count(this\ is)}
$$

In [78]:
def ngram(documents, N=2):
    
    #建立儲存預測字, 所有ngram詞頻字典, 所有字詞(分母)
    ngram_pred = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob']) #使用namedtuple來儲存預測字詞與對應機率

    for doc in documents:
        # 在每個文章前加上起始(<start>)與結束符號(<end>)
        w_list = ['<start>'] + list(doc) + ['<end>']
        
        # 計算分子
        # 若２字一組做預測，總組數＝len(x)-1，最後１個字會落單所以不能計入
        # 若３字一組做預測，總組數＝len(x)-2，最後２個字會落單所以不能計入
        
        for i in range(len(w_list)-N+1):
            total_grams.append(tuple(w_list[i:i+N])) # eg. w_list[0:2]＝挑選最開頭２字
        
        # 計算分母
        # 若２字一組做預測，分母為１字，總組數＝len(x)，無落單字
        # 若３字一組做預測，分母為２字，總組數＝len(x)-1，最後１個字會落單所以不能計入
        
        for i in range(len(w_list)-N+2):
            words.append(tuple(w_list[i:i+N-1]))
    
    #計算分子詞頻
    total_word_counter = Counter(total_grams)
    #計算分母詞頻
    word_counter = Counter(words)
    
    #計算所有N-gram預測字詞的機率
    for key in total_word_counter:
        word = ''.join(key[:N-1])
        if word not in ngram_pred:
            ngram_pred.update({word: set()}) # two_gram_pred = {'桃': set()}

        next_word_prob = total_word_counter[key]/word_counter[key[:N-1]]
        w = Word(key[-1], next_word_prob) # w = Word(word='園', prob=0.92)
        ngram_pred[word].add(w)
        
    return ngram_pred

In [100]:
total_word_counter[('桃', '園')] # Counter{(('桃', '園'), 6607), ()....}

6607

In [105]:
word_counter[('桃',)] # Counter{(('桃',), 7178), ()....}

7178

### 測試 two_gram_pred

In [76]:
two_gram_pred = dict()
Word = namedtuple('Word', ['word', 'prob']) 
# 使用namedtuple來儲存預測字詞與對應機率
# 宣告一種名稱(Word)且包含兩個欄位(word, prob)在一個list中

for key in total_word_counter:
    word = ''.join(key[:2-1])
    if word not in two_gram_pred:
        two_gram_pred.update({word: set()}) # two_gram_pred = {'桃': set()}
    
    next_word_prob = total_word_counter[key]/word_counter[key[:2-1]]
    w = Word(key[-1], next_word_prob) # w = Word(word='園', prob=0.92)
#     w = Word(key[-1], f'{next_word_prob}')# w = Word(word='園', prob='0.92')
    two_gram_pred[word].add(w)

print('排列前:\n', list(two_gram_pred['桃'])[:3])

排列前:
 [Word(word='機', prob=0.0018110894399554194), Word(word='相', prob=0.00013931457230426304), Word(word='警', prob=0.00027862914460852607)]


In [77]:
for word, pred in two_gram_pred.items():
    two_gram_pred[word] = sorted(pred, key=lambda x: x.prob, reverse=True)

print('排列後:\n', list(two_gram_pred['桃'])[:3])

排列後:
 [Word(word='園', prob=0.9204513792142658), Word(word='捷', prob=0.022708275285594874), Word(word='市', prob=0.010866536639732517)]


---
### 使用N-Gram模型進行預測
這裡我們使用4 gram模型，也就是輸入三個字之後，第四個字出現的機率，並將輸出依據機率做排序

In [79]:
#建立four_gram模型，並將預測的機率按照大小排列
four_gram_pred = ngram(documents=corpus_list, N=4)
for word, pred in four_gram_pred.items():
    four_gram_pred[word] = sorted(pred, key=lambda p: p.prob, reverse=True)

In [87]:
# 執行時因為字典數龐大，顯示會較慢
type(four_gram_pred)

dict

In [81]:
#給定字詞，使用ngram預測下一個字的機率(顯示top 10)
text = '鄭文燦'
next_words = list(four_gram_pred[text])[:10]
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 今, probability: 0.9795698924731183
next word: 市, probability: 0.00967741935483871
next word: 模, probability: 0.002150537634408602
next word: 表, probability: 0.002150537634408602
next word: 於, probability: 0.001075268817204301
next word: 率, probability: 0.001075268817204301
next word: 台, probability: 0.001075268817204301
next word: 成, probability: 0.001075268817204301
next word: 回, probability: 0.001075268817204301
next word: 昨, probability: 0.001075268817204301


---
### 可自行嘗試使用不同的N搭建不同的N-Gram模型的效果

In [82]:
# 建立 trigram 模型，並將預測的機率按照大小排列
tri_gram_pred = ngram(documents=corpus_list, N=3)
for word, pred in tri_gram_pred.items():
    tri_gram_pred[word] = sorted(pred, key=lambda p: p.prob, reverse=True)

In [84]:
# 給定字詞，使用 ngram 預測下一個字的機率 (顯示 Top 10)
text = '鄭文'
next_words = list(tri_gram_pred[text])[:10]
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 燦, probability: 1.0


In [85]:
# 給定字詞，使用 ngram 預測下一個字的機率 (顯示 Top 10)
text = '爭取'
next_words = list(tri_gram_pred[text])[:10]
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 中, probability: 0.12680115273775217
next word: 經, probability: 0.05475504322766571
next word: 前, probability: 0.05187319884726225
next word: 更, probability: 0.03170028818443804
next word: 預, probability: 0.02881844380403458
next word: 相, probability: 0.025936599423631124
next word: 到, probability: 0.023054755043227664
next word: 國, probability: 0.020172910662824207
next word: 最, probability: 0.01440922190201729
next word: 桃, probability: 0.01440922190201729


### 建立直接產出預測結果的 function

In [96]:
def result(N, text: str):
    ngram_model = ngram(corpus_list, N) # but每次都要建立一次字典會很慢

    for word, pred in ngram_model.items():
        ngram_model[word] = sorted(pred, key=lambda p: p.prob, reverse=True)
    
    next_words = list(ngram_model[text])[:10]
    for next_word in next_words:
        print('next word: {}, probability: {}'.format(next_word.word, round(next_word.prob, 5)))

In [97]:
result(3, '爭取')

next word: 中, probability: 0.1268
next word: 經, probability: 0.05476
next word: 前, probability: 0.05187
next word: 更, probability: 0.0317
next word: 預, probability: 0.02882
next word: 相, probability: 0.02594
next word: 到, probability: 0.02305
next word: 國, probability: 0.02017
next word: 最, probability: 0.01441
next word: 桃, probability: 0.01441


In [98]:
result(3, '市府')

next word: 也, probability: 0.10018
next word: 將, probability: 0.09418
next word: 團, probability: 0.05879
next word: 會, probability: 0.03499
next word: 已, probability: 0.03479
next word: 積, probability: 0.0234
next word: 在, probability: 0.0168
next word: 推, probability: 0.0156
next word: 衛, probability: 0.015
next word: 與, probability: 0.0148


In [99]:
result(4, '潘縣長')

next word: 及, probability: 0.25
next word: 也, probability: 0.25
next word: 在, probability: 0.25
next word: 則, probability: 0.25
